In [ ]:
import cv2  # Import OpenCV library for image processing
from cvzone.HandTrackingModule import HandDetector  # Import HandDetector from cvzone library
import numpy as np  # Import NumPy library for numerical operations
import math  # Import math library for mathematical functions
import time  # Import time library for time-related functions



In [ ]:
# Initialize video capture from default webcam and create HandDetector object with maxHands set to 1
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

In [ ]:
# Define variables for cropping and saving images
offset = 20  # Offset for cropping hand regions
imgSize = 300  # Desired size for cropped images
folder = "Data/stop"  # Folder path to save images
counter = 0  # Counter to keep track of saved images

In [ ]:
# Main loop to capture and process webcam feed
while True:
    # Read a frame from the webcam feed
    success, img = cap.read()
    
    # Detect hands in the frame
    hands, img = detector.findHands(img)

In [ ]:
# If hands are detected, process them
    if hands:
        hand = hands[0]  # Get information about the first detected hand
        x, y, w, h = hand['bbox']  # Extract bounding box coordinates
        #x- The x-coordinate of the top-left corner of the bounding box.
        #y- The y-coordinate of the top-left corner of the bounding box.
        #w-The width of the bounding box.
        #h-The height of the bounding box.
        
        # Ensure hand region is within bounds
        x1, y1 = max(0, x - offset), max(0, y - offset)
        x2, y2 = min(img.shape[1], x + w + offset), min(img.shape[0], y + h + offset)
        imgCrop = img[y1:y2, x1:x2]  # Crop hand region from the frame

In [ ]:
 # If cropped image is not empty, resize and save it
        if imgCrop.size != 0:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255  # Create white background image
            imgCropShape = imgCrop.shape  # Get shape of cropped image
            aspectRatio = h / w  # Calculate aspect ratio
            
            # Resize cropped image based on aspect ratio
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize


In [ ]:
 # Display cropped and resized image
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)

In [ ]:
# Save image when 's' key is pressed
            key = cv2.waitKey(1)
            if key == ord("s"):
                counter += 1
                cv2.imwrite(f'{folder}/Image_{time.time()}.jpg', imgWhite)  # Save image with unique filename
                print(f"Saved Image {counter}")

In [ ]:
 # Display total number of saved images on the original frame
    cv2.putText(img, f"Saved Images: {counter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow("Image", img)  # Display original frame

In [ ]:
 # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
